# Demonstrates the working of a sequential chain

Create a Sequential Chain that takes 3 variables (A, B and C).

**Input**
* Three words {A}, {B}, {C}

**Chain# 1:**  
* Creates a partial sentence starting with {A}. The size of the partial sentence = 3
* Creates output variable with name = *output_1*

**Chain# 2:**  
* Extends the partial sentence from chain#1 by using {B}
* Creates output variable with name = *output_2*

**Chain# 3**
* Completes the sentence using the word {C}
* Creates output variable with name = *output_3*

**Output**
* Three words {A}, {B}, {C}, {output_1}, {output_2}, {output_3}

In [1]:
from dotenv import load_dotenv
import os
import warnings
from IPython.display import JSON

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

#### Select LLM to use

**NOTE**

To try out different LLMs, call appropriate function below.

* You may see different or even incorrect results
* Adjust the prompts as needed




In [2]:
from langchain_community.llms import Cohere
from langchain.llms import HuggingFaceHub
from langchain_openai import ChatOpenAI

### Cohere command
def create_cohere_command():
    return Cohere(model="command", max_tokens=3000)

### HuggingFace model
def  create_hugging_face():
    
    llm_hf = HuggingFaceHub(
        repo_id='google/flan-t5-xxl', 
        model_kwargs={ "temperature": "0.9" }
    )
    
    ## We need this to warm up the model :)
    print(llm_hf.invoke("hello model"))
    return llm_hf

### Open AI
def create_openai_gpt():
    return ChatOpenAI(temperature=0.4) #model_name=model_name)


### Change function name to try out different provider or model
llm = create_openai_gpt()

## Chain 1

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains  import LLMChain, SequentialChain

template_1 = """
Use the word '{A}' as the start of a partial sentence. 

Predict two words that will come after '{A}'

Your output will be in the following format.

Sentence:
Put the partial sentence here.
"""

In [5]:
prompt_template_1 = PromptTemplate(
                        template=template_1,
                        input_variables = ["A", "B", "C"]
                    )

llm_chain_1 = LLMChain(
    prompt = prompt_template_1,
    llm = llm,
    output_key = 'output_1',
)

## Chain 2

In [6]:
template_2 = """
Extend the following partial sentence by using the word {B}. 

Partial Sentence:
{output_1}

Your output will be in the following format.

Sentence:
Put the partial sentence here.
"""

prompt_template_2 = PromptTemplate(
                        template=template_2,
                        input_variables = ["B","output_1"]
                    )

llm_chain_2 = LLMChain(
    prompt = prompt_template_2,
    llm = llm,
    output_key = 'output_2',
)

## Chain 3

In [7]:
template_3 = """
Complete the following partial sentence by using the word '{C}'. 

Partial Sentence:
{output_2}

Your output will be in the following format.

Sentence:
Put the complete sentence here.
"""

prompt_template_3 = PromptTemplate(
                        template=template_3,
                        input_variables = ["C","output_2"]
                    )

llm_chain_3 = LLMChain(
    prompt = prompt_template_3,
    llm = llm,
    output_key = 'output_3',
)

## Sequential chain

In [8]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(
                    chains = [llm_chain_1, llm_chain_2, llm_chain_3],
                    input_variables = ["A", "B", "C"],
                    output_variables = ["A", "B", "C", "output_1", "output_2", "output_3"]
)

## Infrence

In [9]:
response = sequential_chain.invoke({"A":"power", "B":"energy", "C":"cities"})

In [10]:
JSON(response)

<IPython.core.display.JSON object>

In [11]:
llm

HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-xxl', timeout=None)>, repo_id='google/flan-t5-xxl', task='text2text-generation', model_kwargs={'temperature': '0.9'})